# Speech Understanding 
# Lecture 14: Personal Assistant


### Mark Hasegawa-Johnson, KCGI

1. <a href="#section1">What time is it?</a>
1. <a href="#section2">Tell me a joke!</a>
1. <a href="#section3">Show me my calendar for today</a>
1. <a href="#section4">Personal assistant</a>
1. <a href="#homework">Homework</a>


<a id='section1'></a>

## 1. What time is it?

We will use the `datetime` package to find out what time it is.  We will get the date and time in ISO standard format, then parse them, and read the result back to the user.  First, let's see what the ISO standard format looks like.

In [12]:
import datetime

print(datetime.datetime.now().isoformat())

2024-07-22T23:56:53.688885


You can see that the ISO standard format has the date, followed by the the hour, minutes, and seconds.  Our personal assistant will only read to us the hour and minutes.  We can use `split` to split the string at the `T` character, and then to split it at every `:` character, then read out only the relevant part:


In [13]:
(date, time) = datetime.datetime.now().isoformat().split("T")

(hour, minutes, seconds) = time.split(":")

print(hour+"時"+minutes+"分です")

23時56分です


Let's create a function that tells us the current time.

In [14]:
import datetime, gtts

def what_time_is_it(lang, filename):
    '''
    Tell me what time it is.
    
    Parameters:
    lang (str) - language in which to speak
    filename (str) - the filename into which the audio should be recorded
    '''
    (date, time) = datetime.datetime.now().isoformat().split("T")
    (hour, minutes, seconds) = time.split(":")
    if lang=="en":
        text = hour+" hours and "+minutes+" minutes"
    elif lang=="ja":
        text = hour+"時"+minutes+"分です"
    elif lang=="zh":
        text = "现在是"+hour+"点"+"分"
    else:
        text="I'm sorry, I don't know that language"
    gtts.gTTS(text,lang=lang).save(filename)
 

If you've created that file, you can now test it by running the following block:

In [15]:
what_time_is_it("ja", "time.mp3")

import librosa, IPython
x, fs = librosa.load('time.mp3')
IPython.display.Audio(data=x, rate=fs)

<a id='section2'></a>

## 2. Tell me a joke!

If you want your personal assistant to tell jokes, you need a good source of jokes.  There are many lists of jokes on the internet, though most of the jokes are not very funny.

### 2.1 Jokes in English

In English, let's use this list of jokes from the `yesinteractive/dadjokes` app:  https://raw.githubusercontent.com/yesinteractive/dadjokes/master/controllers/jokes.txt

In [16]:
import requests
req = requests.get("https://raw.githubusercontent.com/yesinteractive/dadjokes/master/controllers/jokes.txt")

texts = {}
texts['en'] = req.text
with open('jokes_en.txt','w') as f:
    f.write(texts['en'].replace('<>','  '))
    
jokes = {}
jokes['en'] = []
with open('jokes_en.txt', 'r') as f:
    jokes['en'] = f.readlines()

for n in range(6):
    print(jokes['en'][n])

What did one pirate say to the other when he beat him at chess?  Checkmatey.

I burned 2000 calories today  I left my food in the oven for too long.

I startled my next-door neighbor with my new electric power tool.   I had to calm him down by saying “Don’t worry, this is just a drill!”

I broke my arm in two places.   My doctor told me to stop going to those places.

I quit my job at the coffee shop the other day.   It was just the same old grind over and over.

I never buy anything that has Velcro with it...  it’s a total rip-off.



### 2.2 Jokes in Japanese

Here is a web page containing some jokes in Japanese: http://www.gujo-tv.ne.jp/~circleband/oyaji%20gag.htm.  

Open the page source, and look at `content` at the top.  Notice that this file is stored in `Shift_JIS`, which is a pre-unicode encoding for Japanese characters.  So the first thing we need to do is to convert it to unicode.  One way to do that is by saving the raw binary to a file, and then reading it back in using the `Shift_JIS` decoder:


In [17]:
import requests

rawbinary = requests.get("http://www.gujo-tv.ne.jp/~circleband/oyaji%20gag.htm").content
with open('jokes_ja.htm','wb') as f:
    f.write(rawbinary)
    
with open('jokes_ja.htm', 'r', encoding='shiftjis') as f:
    texts['ja'] = f.read()

print(texts['ja'][0:1000])


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<HTML>
<HEAD>
<META name="GENERATOR" content="IBM WebSphere Homepage Builder V6.0.1 for Windows">
<META http-equiv="Content-Type" content="text/html; charset=Shift_JIS">
<META http-equiv="Content-Style-Type" content="text/css">
<TITLE>おやじギャグ</TITLE>
</HEAD>
<BODY text="#FFFF00" link="#00FFFF" vlink="#00FF00" background="st09_bg.gif">
<P align="center"><IMG alt="直線上に配置" width="512" height="50" border="0" src="st09_l1.gif"><FONT color="#00cccc"><B><FONT size="+3"><BR>
おやじギャグ</FONT></B></FONT></P>
<CENTER>
<TABLE border="1">
  <TBODY>
    <TR>
      <TD align="center">1</TD>
      <TD align="center">１年生</TD>
      <TD align="center">トマトを食べるの　ちょっと待っとって</TD>
    
    <TR>
      <TD align="center">2</TD>
      <TD align="center">１年生</TD>
      <TD align="center">お金を取られた　おっかねー</TD>
    
    <TR>
      <TD align="center">3</TD>
      <TD align="center">１年生</TD>
      <TD align="center">スイカを積んだ　せんすいかん</TD>
    
    <TR>
      <TD a

Notice that each joke is the third `<td>` tag under a `<tr>` tag.  Let's use BeautifulSoup to find those.


In [18]:
import bs4
soup = bs4.BeautifulSoup(texts['ja'], "html.parser")

jokes['ja'] = []
for tr in soup.find_all('tr'):
    tdlist = tr.find_all('td')
    jokes['ja'].append(tdlist[2].text + '\n')

for n in range(5):
    print(jokes['ja'][n])

with open('jokes_ja.txt','w') as f:
    f.writelines(jokes['ja'])

トマトを食べるの　ちょっと待っとって

お金を取られた　おっかねー

スイカを積んだ　せんすいかん

トイレでバッタが　ふんばった

このアジ　とっても味がある



UnicodeEncodeError: 'gbk' codec can't encode character '\u30fb' in position 7: illegal multibyte sequence

### 2.3 Jokes in Chinese

As in Japanese, we need to somehow extract jokes from a web page.  Let's try this web page:  http://www.ziyexing.com/files_8/xiaohua_01.htm.  This file is encoded in gb2312, so we will have to convert it to Unicode, just as we did with the Japanese jokes.

In [19]:
import requests

rawbinary = requests.get("http://www.ziyexing.com/files_8/xiaohua_01.htm").content
    
with open('jokes_zh.htm','wb') as f:
    f.write(rawbinary)
    
with open('jokes_zh.htm', 'r', encoding="gb2312", errors="ignore") as f:
    texts['zh'] = f.read()

print(texts['zh'][:2000])


<html>

<head>
<meta http-equiv="Content-Type" content="text/html; charset=gb2312">
<title>百花园・笑话集锦〔一〕　-- 子夜星网站</title>
<link href="xiaohua.css" rel="stylesheet" type="text/css">
</head>
<body bgcolor="#F2F3FB">
<div align="center">
  <center>
			<tr>
			<td width="100%">
			<div align="center">
			<center>
			<table id="table1">
			<tr>
			<td width="100%">
			<div align="center">
			<table border="0" cellpadding="0" cellspacing="0" width="98%">
			<tr>
			<td width="70%"><span style="font-size: 10pt">　<a target="_blank" href="http://www.ziyexing.com/">首页</a> &gt;&gt; <a target="_blank" href="baihuayuan_index.htm">百花园</a> &gt;&gt;
			<a href="xiaohua_01.htm">笑话集锦</a> &lt;1&gt;</span></td>
			<td width="30%">
			<img border="0" src="../material/midnight.gif" align="right" width="167" height="14"></td>
			</tr>
			</table>
		</div>
			</td>
			</tr>
			<tr>
			<td id="td_01">　</td>
			</tr>
			<tr>
			<td width="100%">
			<div align="center">
			<table id="table2">
			<tr>
			<td>
			<

* All of the jokes are in one long paragraph.  This paragraph is distinguished from other paragraphs only because it has `style="line-height: 150%"`
* Each joke begins with a line in boldface (`<b>...</b>`), starting with the character `○`.  We can probably just discard the `<b>` and `</b>` tags using the python string `replace` function, and then split the paragraph at the `○` character.
* Each joke contains several lines, with lots of extra whitespace.  We can split on `\n` characters, eliminate `<br>` tags, and then use `strip` to get rid of extra whitespace.


In [20]:
import bs4
soup = bs4.BeautifulSoup(texts['zh'], "html.parser")

ptag = soup.find("p",{'style':'line-height: 150%'})
jokestext = ptag.text.replace('</b>','').replace('<br>','').replace('<b>','')

jokes['zh'] = []
for rawjoke in jokestext.split('○'):
    lines = rawjoke.split('\n')
    joke = lines[0].strip() + ': '
    if len(lines) > 1:
        for line in lines[1:]:
            joke += line.strip()
        jokes['zh'].append(joke + '\n')

for n in range(10):
    print('Joke number ',n,': ', jokes['zh'][n])

with open('jokes_zh.txt','w') as f:
    f.writelines(jokes['zh'])

Joke number  0 :  丈夫回家见媳妇在揍孩子: 丈夫回家，见妻子在揍孩子，忍了忍没理她，走到厨房看见桌上煮好一锅馄饨，于是吃了一碗。吃完见老婆还在揍儿子，终于忍不住了：“教育小孩不能老用暴力，要多讲道理嘛！”妻子说：“好好的一锅馄饨，他居然撒了一泡尿进去，你说气人不气人。”丈夫听后马上说：“媳妇你歇一会，让我来揍！”

Joke number  1 :  姐夫的车牌号00544: 一乡镇企业领导中途遇见了小姨子，赶紧刹车。小姨子：“姐夫又换新车了，够威风啊！00544，车牌号也换啦？”姐夫：“这个车牌号才硬呢，意思是──动动我试试！”某日，这位乡镇企业领导因车祸住进了医院。小姨子来病房探视，见到姐夫头缠绷带、还吊着腿的窘相，不禁扑哧一声笑了。小姨子：“你不是说你的车牌号硬吗，咋弄成这模样啦？”姐夫：“妈的，那个肇事车的牌号比我的还硬：44944 ！”

Joke number  2 :  院长跳楼: 医院院长因股市暴跌而跳楼，好不容易抢救过来。家人纷纷围在床前，问他想要什么。院长虚弱地道：“我…只想要…沪…市…涨！”老婆一耳光过去：“我一直怀疑你和护士长有关系，到死还想着她！”

Joke number  3 :  朋友戒烟: 一友总想戒烟，总是戒不掉。一日途中犯瘾，掏出火柴点烟。由于风大，点了两次都没点着，便念念有词道：“点烟不过三，过三不吸烟。”结果第三次还没点着，索性把烟放回兜中，不抽了。可一会儿又掏了出来，嘴里又念叨着：“点烟不过七，过七我不吸。”未料到七次又没点着，便急了：“管他三七二十一，啥时点着啥时吸！”

Joke number  4 :  测谎: 某领导儿子爱说谎，领导买了台测谎机器人。一日儿子晚归，父问：“去哪了？”答：“图书馆看书。”机器人一巴掌拍了过去。儿不得不说：“去同学家看黄片了。”父大怒：“好大胆子，我长这么大都还没看过呢！”啪！机器人给了其父一巴掌。妻子怒斥丈夫：“活该！对儿子这么苛刻，不管怎么说他都是你亲生的啊！”啪！机器人又给了妻子一耳光。

Joke number  5 :  打官司: 一位移居海外的原中国官员，在美国摊上了官司。他与自己的律师商量：“是否可以用钱打点一下法官和陪审员们？”律师说：“若那样，你就死定了。”这位原中国官员心领神会，便雇人冒充对方当事人关系，对法官和陪审员行贿。于是，他赢了。

Joke

### 2.4 Tell me a joke!

Let's ask gtts to tell us a joke chosen at random from one of these lists.  Add the following to your `week14.py` file:

In [21]:
import gtts, bs4, random

def tell_me_a_joke(lang, audiofile):
    '''
    Tell me a joke.
    
    @params:
    filename (str) - filename containing the database of jokes
    lang (str) - language
    audiofile (str) - audiofile in which to record the joke
    '''
    filename = 'jokes_%s.txt'%(lang)
    with open(filename) as f:
        jokes = f.readlines()
    joke = random.choice(jokes)
    print(joke.strip())
    gtts.gTTS(joke.strip(), lang=lang).save(audiofile)


Let's try it:

In [22]:
tell_me_a_joke("ja", 'joke.mp3')

import librosa, IPython
x, fs = librosa.load('joke.mp3')
IPython.display.Audio(data=x, rate=fs)

猫が寝込んだ　サルが去る


<a id='section3'></a>

## 3. What day is it today?

Let's ask our assistant to tell us today's date.  In order to give her something to say, we will also ask her to say what is today's current date, using the python `datetime` package.  
`datetime` gives the current date in several different ways.  For example,

In [23]:
import datetime
print('The year is',datetime.date.today().year)
print('The month is',datetime.date.today().month)
print('The day is',datetime.date.today().day)
print('It is the',datetime.date.today().isoweekday(),'day of the week')

The year is 2024
The month is 7
The day is 23
It is the 2 day of the week


We can use some dictionaries to make those numbers more friendly.

In [24]:
def what_day_is_it(lang, audiofile):
    '''
    Tell me what day it is.

    @params:
    lang (str) - language in which to record the date
    audiofile (str) - filename in which to read the date
    
    @returns:
    url (str) - URL that you can look up in order to see the calendar for this month and year
    '''
    today = datetime.date.today()
    year = today.year
    month = today.month
    day = today.day
    weekday = today.isoweekday()
    if lang=="en":
        weekdays=['','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
        months=['','January','February','March','April','May','June','July','August','September','October','November','December']
        text = "%s, %s %d, %d"%(weekdays[weekday],months[month],day,year)
        gtts.gTTS("Today is "+text,lang="en").save(audiofile)
    elif lang=="ja":
        weekdays=' 月火水木金土日'
        text="%s曜日,%d月%d日, %d年"%(weekdays[weekday],month,day,year)
        gtts.gTTS("今日は"+text,lang="ja").save(audiofile)
    elif lang=="zh":
        weekdays=['','周一','周二','周三','周四','周五','周六','星期日']
        text='%s, %d月%d日, %d年'%(weekdays[weekday],month,day,year)
        gtts.gTTS("今天是"+text,lang="zh").save(audiofile)


Let's try it:

In [25]:
what_day_is_it("zh","date.mp3")

import librosa, IPython
x, fs = librosa.load('date.mp3')
IPython.display.Audio(data=x, rate=fs)

<a id='section4'></a>

## 4. Personal assistant

Now let's put it all together in a personal assistant app.  Your personal assistant will listen to you, and respond when you make any of the following four types of requests:

* If you say anything containing "What time," it will tell you the time
* If you say anythin containing the word "joke," it will tell you a joke
* If you say anything containing the word "calendar," it will open your calendar
* If you say something else, it will say "I'm sorry, I didn't understand you!"

First, let's create a convenience function that listens to us:

In [26]:
import speech_recognition

def personal_assistant(lang, filename):
    if lang=="en":
        keywords = ["what time", "joke", "what day", "I'm sorry, I didn't understand you"]
    elif lang=="ja":
        keywords = ["何時","冗談","何日","すみません、よくわかりませんでした"]
    elif lang=="zh":
        keywords = ["几奌","玩笑","什么日子","对不起，我没听懂你的话"]
    else:
        speech_package.synthesize("I don't know that language!","en",filename)
        return

    r = speech_recognition.Recognizer()

    while True:
        print('Listening...')
        with speech_recognition.Microphone() as source:
            r.adjust_for_ambient_noise(source)
            try:
                audio = r.listen(source)
                text = r.recognize_google(audio, language=lang)
            except speech_recognition.UnknownValueError:
                print('I did not understand that, I will try again')
                continue
            except sr.RequestError:
                print('Sorry, I could not reach the internet, I will try again')
                continue
            except sr.WaitTimeoutError:
                continue
            
        print("I heard",text)
        if keywords[0] in text:
            what_time_is_it(lang, filename)
            break
        elif keywords[1] in text:
            tell_me_a_joke(lang, filename)
            break
        elif keywords[2] in text:
            what_day_is_it(lang, filename)
        else:
            print(keywords[3])
            print('I will try again')
            

Now let's try running the personal assistant:

In [ ]:
personal_assistant("ja", "test.mp3")

x, fs = librosa.load("test.mp3")
IPython.display.Audio(data=x, rate=fs)

## Homework

In the homework file `homework14.py`, please create four functions `what_time_is_it`, `tell_me_a_joke`, `what_day_is_it`, and `personal_assistant` like those above.

In [1]:
import homework14, importlib
importlib.reload(homework14)
help(homework14.what_time_is_it)

Help on function what_time_is_it in module homework14:

what_time_is_it(lang, filename)
    Tell me what time it is.
    
    Parameters:
    lang (str) - language in which to speak
    filename (str) - the filename into which the audio should be recorded



In [2]:
help(homework14.tell_me_a_joke)

Help on function tell_me_a_joke in module homework14:

tell_me_a_joke(lang, audiofile)
    Tell me a joke.
    
    Parameters:
    lang (str) - language
    audiofile (str) - audio file in which to record the joke



In [3]:
help(homework14.what_day_is_it)

Help on function what_day_is_it in module homework14:

what_day_is_it(lang, audiofile)
    Tell me what day it is.
    
    Parameters:
    lang (str) - language in which to record the date
    audiofile (str) - filename in which to read the date
    
    Returns:
    url (str) - URL that you can look up in order to see the calendar for this month and year



In [4]:
help(homework14.personal_assistant)

Help on function personal_assistant in module homework14:

personal_assistant(lang, filename)
    Listen to the user, and respond to one of three types of requests:
    What time is it?
    What day is it?
    Tell me a joke!
    
    Parameters:
    lang (str) - language
    filename (str) - filename in which to store the result



When everything is working, you should be able to run personal_assistant from your homework file:

In [5]:
importlib.reload(homework14)
homework14.personal_assistant('ja','test.mp3')
x, fs = librosa.load("test.mp3")
IPython.display.Audio(data=x, rate=fs)

What would you like me to do?tell me a joke
I heard tell me a joke
すみません、よくわかりませんでした
I will try again


NameError: name 'librosa' is not defined

### Receiving your grade

In order to receive a grade for your homework, you need to:

1. Run the following code block on your machine.  The result may list some errors, and then in the very last line, it will show a score.  That score (between 0% and 100%) is the grade you have earned so far.  If you want to earn a higher grade, please continue editing `homework3.py`, and then run this code block again.
1. When you are happy with your score (e.g., when it reaches 100%), choose `File` $\Rightarrow$ `Save and Checkpoint`.  Then use `GitHub Desktop` to commit and push your changes.
1. Make sure that the 100% shows on your github repo on github.com.  If it doesn't, you will not receive credit.

In [6]:
import importlib, grade
importlib.reload(grade)

..

Error reading the joke file. Please check the file encoding.
Error reading the joke file. Please check the file encoding.


....
----------------------------------------------------------------------
Ran 6 tests in 2.342s

OK
..

6 successes out of 6 tests run
Score: 100%
Error reading the joke file. Please check the file encoding.
Error reading the joke file. Please check the file encoding.


....
----------------------------------------------------------------------
Ran 6 tests in 2.255s

OK


6 successes out of 6 tests run
Score: 100%


<module 'grade' from 'C:\\Users\\TT\\Documents\\GitHub\\intro_speech_understanding\\2024_spring\\lec14\\grade.py'>